In [1]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import funcionalidade as fc

In [2]:
ativos = pd.read_pickle('ativos_fatura_premiada.pkl')

In [3]:
#ativos.to_csv('ativos_fatura_premiada.csv')

In [4]:
ativos.head(3)

,cpf,grupo_comunicacao,segmento_mcc_agg,nr_score_ie,yearmonth_ie,status,safra
0,91908973234,Comunicação MCC,1 - SUPERMERCADO,29.713168,202302.0,Ativo,2021-09-01
1,14258158470,Comunicação MCC,1 - SUPERMERCADO,50.899609,202302.0,Ativo,2021-10-01
2,22377212468,Comunicação MCC,1 - SUPERMERCADO,59.104737,202302.0,Ativo,2022-02-01


In [3]:
ativos['grupo_comunicacao'].value_counts()

Comunicação MCC         1404826
Sem comunicação          173079
Comunicação Genérica     150575
Name: grupo_comunicacao, dtype: int64

In [5]:
#ativos['fx_ie'] = pd.qcut(ativos['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])

In [9]:
#ativos[['fx_ie', 'nr_score_ie']].groupby('fx_ie').agg({'nr_score_ie':['min','max']})
                                                       

nr_score_ie            
              min         max
fx_ie                        
baixo    0.000000   44.745763
medio   44.745763   62.181660
alto    62.181660  100.000000

## ATIVOS - Análise comunicação genérica x MCC

In [4]:
ativos_v1 = ativos.query('grupo_comunicacao != "Sem comunicação"')
ativos_v1['grupo'] = ['alvo' if x == 'Comunicação MCC' else 'controle' for x in ativos_v1['grupo_comunicacao']]

/tmp/ipykernel_177885/1857667637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ativos_v1['grupo'] = ['alvo' if x == 'Comunicação MCC' else 'controle' for x in ativos_v1['grupo_comunicacao']]


### Trazendo conversão

In [5]:
ativos_v1_c = fc.traz_conversao('flavia-costa', '2023-03-17', '2023-03-24', 'cpf', ativos_v1)

In [6]:
len(ativos_v1_c)

1555401

In [9]:
ativos_v1_c['converteu'].value_counts()

0.0    796180
1.0    759221
Name: converteu, dtype: int64

In [10]:
ativos_v1_c.head(3)

,cpf,grupo_comunicacao,segmento_mcc_agg,nr_score_ie,yearmonth_ie,status,safra,grupo,conta_cartao,converteu,transacoes,spending,spending_corr,transacoes_corr
0,91908973234,Comunicação MCC,1 - SUPERMERCADO,29.713168,202302.0,Ativo,2021-09-01,alvo,NaN,0.0,NaN,NaN,0.0,0.0
1,14258158470,Comunicação MCC,1 - SUPERMERCADO,50.899609,202302.0,Ativo,2021-10-01,alvo,NaN,0.0,NaN,NaN,0.0,0.0
2,22377212468,Comunicação MCC,1 - SUPERMERCADO,59.104737,202302.0,Ativo,2022-02-01,alvo,NaN,0.0,NaN,NaN,0.0,0.0


### Traz personas

In [11]:
import acessos as ac

In [12]:
personas = ac.df_athena_q('flavia-costa', 
                          """select 
                                cpf
                                , max(ds_cluster_sociodemographic) as ds_cluster_sociodemographic
                                from customer_curated_zone.ca_book_cliente_cluster cl
                                left join customer_curated_zone.ca_book_cliente ci on (ci.id_customer = cl.id_customer)
                                group by 1 """)

ativos_v1_p = ativos_v1_c.join(personas.set_index('cpf'), how = 'left', on = 'cpf')

### Análises de Lift

Análise reduzida geral

In [13]:
fc.testes_iterativos(ativos_v1_c)

            grupo converteu    spending               transacoes           
            count      mean        mean           sum       mean        sum
grupo                                                                      
alvo      1404826  0.488066  337.802139  2.316134e+08   3.467613  2377562.0
controle   150575  0.488614  330.797051  2.433773e+07   3.442363   253265.0
_____________________
Lift da conversão: -0.11%
Estatística t do teste de diferença de proporções: -0.4
P-valor do teste de diferença de proporções: 0.686261
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 2.12%
Estatística t do teste de spending: 2.68
P-valor do teste de spending: 0.007371
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 0.73%
Estatística t do teste de frequencia: 1.3
P-valor do teste de frequencia: 0.194583

In [7]:
fc.testes_iterativos(ativos_v1_c.query('segmento_mcc_agg != "0 - OUTROS"'))

           grupo converteu    spending               transacoes           
           count      mean        mean           sum       mean        sum
grupo                                                                     
alvo      772563  0.566508  336.351192  1.472085e+08   3.735040  1634689.0
controle   85876  0.566398  336.265502  1.635595e+07   3.702426   180086.0
_____________________
Lift da conversão: 0.02%
Estatística t do teste de diferença de proporções: 0.06
P-valor do teste de diferença de proporções: 0.950892
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 0.03%
Estatística t do teste de spending: 0.05
P-valor do teste de spending: 0.957661
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 0.88%
Estatística t do teste de frequencia: 1.6
P-valor do teste de frequencia: 0.109622
A diferença de fre

Análise por recortes

In [8]:
fc.testes_iterativos(ativos_v1_c.query('segmento_mcc_agg != "0 - OUTROS"').query('nr_score_ie >47.35 & nr_score_ie <= 51.52'))

          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      54476  0.485278  294.736423  7791652.07   2.820699  74568.0
controle   6059  0.480442  295.890460   861337.13   2.760220   8035.0
_____________________
Lift da conversão: 1.01%
Estatística t do teste de diferença de proporções: 0.71
P-valor do teste de diferença de proporções: 0.474947
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: -0.39%
Estatística t do teste de spending: 0.17
P-valor do teste de spending: 0.866432
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 2.19%
Estatística t do teste de frequencia: 1.33
P-valor do teste de frequencia: 0.182292
A diferença de frequência não é significa

In [10]:
fc.testes_iterativos(ativos_v1_c.query('segmento_mcc_agg == "0 - OUTROS"'))

           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      632263  0.392120  339.724607  84225543.64   2.994438  742390.0
controle   64699  0.385246  319.103925   7953665.33   2.933641   73121.0
_____________________
Lift da conversão: 1.78%
Estatística t do teste de diferença de proporções: 3.41
P-valor do teste de diferença de proporções: 0.000644
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 6.46%
Estatística t do teste de spending: 5.65
P-valor do teste de spending: 0.0
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 2.07%
Estatística t do teste de frequencia: 4.39
P-valor do teste de frequencia: 1.1e-05
A difer

In [9]:
fc.testes_iterativos(ativos_v1_c.query('segmento_mcc_agg == "0 - OUTROS"').query('nr_score_ie >47.35 & nr_score_ie <= 51.52'))

          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      50525  0.437704  321.425816  7108331.92   2.804793  62028.0
controle   5205  0.411527  300.908492   644545.99   2.732493   5853.0
_____________________
Lift da conversão: 6.36%
Estatística t do teste de diferença de proporções: 3.63
P-valor do teste de diferença de proporções: 0.000287
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 6.82%
Estatística t do teste de spending: 2.63
P-valor do teste de spending: 0.008627
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 2.65%
Estatística t do teste de frequencia: 3.08
P-valor do teste de frequencia: 0.002057
A diferença de f

In [14]:
#cria faixa de IE
ativos_v1_c['fx_ie'] = pd.qcut(ativos_v1_c['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
ativos_v1_c['fx_ie'].value_counts()

baixo    514866
medio    514866
alto     514866
Name: fx_ie, dtype: int64

In [15]:
for x in ['baixo', 'medio', 'alto']:
    temp_df = pd.DataFrame()
    print(f"=============Faixa de IE: {x}============")
    temp_df = ativos_v1_c[ativos_v1_c['fx_ie'] == x]
    fc.testes_iterativos(temp_df)

=============Faixa de IE: baixo============
           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      466362  0.332493  330.114156  51188161.28   2.730721  423431.0
controle   48504  0.334735  318.878744   5177315.29   2.689209   43662.0
_____________________
Lift da conversão: -0.67%
Estatística t do teste de diferença de proporções: -1.0
P-valor do teste de diferença de proporções: 0.318485
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 3.52%
Estatística t do teste de spending: 1.48
P-valor do teste de spending: 0.139312
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 1.54%
Estatística t do teste de frequencia: 0.73
P-valor do teste de fre

In [16]:
ativos_v1_c.segmento_mcc_agg.value_counts()

0 - OUTROS                     696962
4 - SAUDE                      238634
3 - SERVICOS DE ALIMENTACAO    200283
2 - LOJAS DE DEPARTAMENTO      195689
1 - SUPERMERCADO               112159
5 - LOJAS ESPECIALIZADAS       111674
Name: segmento_mcc_agg, dtype: int64

In [17]:
#por segmento MCC
for x in ['0 - OUTROS', '4 - SAUDE', '3 - SERVICOS DE ALIMENTACAO','2 - LOJAS DE DEPARTAMENTO','1 - SUPERMERCADO', '5 - LOJAS ESPECIALIZADAS']:
    temp_df = pd.DataFrame()
    print(f"=============Segmento MCC: {x}============")
    temp_df = ativos_v1_c[ativos_v1_c['segmento_mcc_agg'] == x]
    fc.testes_iterativos(temp_df)

=============Segmento MCC: 0 - OUTROS============
           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      632263  0.392156  339.958517  84291354.42   2.994704  742525.0
controle   64699  0.385261  319.488373   7963567.19   2.934045   73134.0
_____________________
Lift da conversão: 1.79%
Estatística t do teste de diferença de proporções: 3.42
P-valor do teste de diferença de proporções: 0.00062
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 6.41%
Estatística t do teste de spending: 5.62
P-valor do teste de spending: 0.0
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 2.07%
Estatística t do teste de frequencia: 4.3

In [18]:
personas_list = ativos_v1_p['ds_cluster_sociodemographic'].unique()
print(personas_list)

['26 - Mulher, 25 a 40 anos, Centro-Oeste e Norte'
 '10 - Mulher, Solteira, Até 24 anos, cidade Pequena do Nordeste'
 '23 - Homem, 41 anos ou mais, Nordeste'
 '15 - Mulher, 25 a 40 anos, Nordeste'
 '19 - Mulher, 41 anos ou mais, Nordeste'
 '24 - Homem, Até 24 anos, Nordeste'
 '11 - Homem, Solteiro, Até 24 anos, cidade Pequena do Nordeste'
 '16 - Homem, 25 a 40 anos, Nordeste'
 '30 - Homem, Até 24 anos, Centro-Oeste e Norte'
 '7 - Homem, Solteiro, 25 anos ou mais, cidade Pequena do Nordeste, ensino fundamental cursando ou completo'
 '18 - Mulher, 25 a 40 anos, Sudeste e Sul'
 '0 - Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste'
 '31 - Mulher, 41 anos ou mais, Centro-Oeste e Norte'
 '25 - Homem, 41 anos ou mais, Sudeste e Sul'
 '14 - Homem, Casado, 35 anos ou mais, cidade Pequena do Nordeste'
 '17 - Homem, 25 a 40 anos, Sudeste e Sul'
 '20 - Homem, Até 24 anos, Sudeste e Sul'
 '28 - Mulher, Até 24 anos, Centro-Oeste e Norte'
 '12 - Mulher, Solteira, 35 anos ou mais, cidade P

In [19]:
for x in personas_list:
    temp_df = pd.DataFrame()
    print(f"=============Personas: {x}============")
    if x is None:
        pass
    else:
        temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'] == x]
        fc.testes_iterativos(temp_df)

=============Personas: 26 - Mulher, 25 a 40 anos, Centro-Oeste e Norte============
          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      43252  0.481273  350.030714  7286239.35   3.558945  74083.0
controle   4713  0.475918  348.217222   781051.23   3.474365   7793.0
_____________________
Lift da conversão: 1.13%
Estatística t do teste de diferença de proporções: 0.7
P-valor do teste de diferença de proporções: 0.48475
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 0.52%
Estatística t do teste de spending: 0.39
P-valor do teste de spending: 0.695631
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 2.43%
Estatística t do teste de frequencia: 1.27
P-

## Analise de combinações

In [20]:
import importlib
importlib.reload(fc)

<module 'funcionalidade' from '/home/flavia.costa/work/codigos/FaturaPremiadaMar2023/funcionalidade.py'>

In [21]:
ativos_v1_p['cont'] = 1

In [25]:
ativos_v1_p['fx_ie'] = pd.qcut(ativos_v1_p['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
ativos_v1_p['fx_ie'].value_counts()

baixo    514866
medio    514866
alto     514866
Name: fx_ie, dtype: int64

In [26]:
colunas_analise = ['ds_cluster_sociodemographic', 'segmento_mcc_agg', 'fx_ie']

ativos_v1_p['fx_ie'] = ativos_v1_p['fx_ie'].astype('string')

report_combinacoes_alvo = fc.report_perfil_grupos(ativos_v1_p.query('grupo == "alvo"'), 'converteu', 'cont', 0.05, 50, colunas_analise)
report_combinacoes_alvo.set_index('segmento', inplace=True)

report_combinacoes_controle = fc.report_perfil_grupos(ativos_v1_p.query('grupo == "controle"'), 'converteu', 'cont', 0.05, 50, colunas_analise)
report_combinacoes_controle.set_index('segmento', inplace=True)

#.query('grupo_comunicacao ==  "Comunicação MCC"')


In [27]:
report_combinacoes = report_combinacoes_alvo.join(report_combinacoes_controle, rsuffix='_c', lsuffix='_t')

In [28]:
report_combinacoes.head()

,n_seg_grupo_t,n_seg_t,n_grupo_t,total_t,per_grupo_seg_t,per_grupo_total_t,diferenca_t,diferenca_abs_t,converteu_t,n_seg_grupo_c,n_seg_c,n_grupo_c,total_c,per_grupo_seg_c,per_grupo_total_c,diferenca_c,diferenca_abs_c,converteu_c
segmento,,,,,,,,,,,,,,,,,,
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste 1 SUPERMERCADO alto",729,1669,719178,1404826,0.436788,0.511934,-0.075145,0.075145,0.0,94,160,73573,150575,0.587500,0.488614,0.098886,0.098886,1.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste 1 SUPERMERCADO alto",729,1669,719178,1404826,0.436788,0.511934,-0.075145,0.075145,0.0,66,160,77002,150575,0.412500,0.511386,-0.098886,0.098886,0.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste 1 SUPERMERCADO alto",940,1669,685648,1404826,0.563212,0.488066,0.075145,0.075145,1.0,94,160,73573,150575,0.587500,0.488614,0.098886,0.098886,1.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste 1 SUPERMERCADO alto",940,1669,685648,1404826,0.563212,0.488066,0.075145,0.075145,1.0,66,160,77002,150575,0.412500,0.511386,-0.098886,0.098886,0.0
"- 0 Mulher, Casada, 25 anos ou mais, cidade Pequena do Nordeste 1 SUPERMERCADO baixo",630,1710,685648,1404826,0.368421,0.488066,-0.119645,0.119645,1.0,104,179,77002,150575,0.581006,0.511386,0.069619,0.069619,0.0


In [29]:
report_combinacoes.query('converteu_t == 1').query('converteu_c == 1').query('diferenca_t > 0').query('diferenca_t - diferenca_c > 0.04').sort_values('diferenca_t', ascending=False)
#.to_csv('temp.csv')


,n_seg_grupo_t,n_seg_t,n_grupo_t,total_t,per_grupo_seg_t,per_grupo_total_t,diferenca_t,diferenca_abs_t,converteu_t,n_seg_grupo_c,n_seg_c,n_grupo_c,total_c,per_grupo_seg_c,per_grupo_total_c,diferenca_c,diferenca_abs_c,converteu_c
segmento,,,,,,,,,,,,,,,,,,
"- 22 Mulher, 41 anos ou mais, Sudeste e Sul 5 LOJAS ESPECIALIZADAS alto",928,1285,685648,1404826,0.722179,0.488066,0.234113,0.234113,1.0,88,131,73573,150575,0.671756,0.488614,0.183142,0.183142,1.0
"- 19 Mulher, 41 anos ou mais, Nordeste 5 LOJAS ESPECIALIZADAS alto",1491,2134,685648,1404826,0.698688,0.488066,0.210622,0.210622,1.0,145,225,73573,150575,0.644444,0.488614,0.155831,0.155831,1.0
"- 31 Mulher, 41 anos ou mais, Centro-Oeste e Norte 4 SAUDE alto",1033,1496,685648,1404826,0.690508,0.488066,0.202442,0.202442,1.0,93,157,73573,150575,0.592357,0.488614,0.103743,0.103743,1.0
"- 26 Mulher, 25 a 40 anos, Centro-Oeste e Norte 5 LOJAS ESPECIALIZADAS alto",1188,1733,685648,1404826,0.685516,0.488066,0.197450,0.197450,1.0,116,198,73573,150575,0.585859,0.488614,0.097245,0.097245,1.0
"- 30 Homem, Até 24 anos, Centro-Oeste e Norte 4 SAUDE alto",1008,1509,685648,1404826,0.667992,0.488066,0.179926,0.179926,1.0,108,172,73573,150575,0.627907,0.488614,0.139293,0.139293,1.0
"- 22 Mulher, 41 anos ou mais, Sudeste e Sul 2 LOJAS DE DEPARTAMENTO alto",844,1272,685648,1404826,0.663522,0.488066,0.175456,0.175456,1.0,96,160,73573,150575,0.600000,0.488614,0.111386,0.111386,1.0
"- 5 Mulher, Casada, 25 anos ou mais, cidade Grande, Capital, do Sudeste Sul 4 SAUDE",634,973,685648,1404826,0.651593,0.488066,0.163527,0.163527,1.0,66,118,73573,150575,0.559322,0.488614,0.070708,0.070708,1.0
"- 23 Homem, 41 anos ou mais, Nordeste 3 SERVICOS DE ALIMENTACAO alto",1579,2438,685648,1404826,0.647662,0.488066,0.159596,0.159596,1.0,165,287,73573,150575,0.574913,0.488614,0.086299,0.086299,1.0
"- 18 Mulher, 25 a 40 anos, Sudeste e Sul 5 LOJAS ESPECIALIZADAS",3763,5816,685648,1404826,0.647008,0.488066,0.158942,0.158942,1.0,393,656,73573,150575,0.599085,0.488614,0.110472,0.110472,1.0


In [30]:
temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("18 - Mulher, 25 a 40 anos, Sudeste")') \
        .query('segmento_mcc_agg.str.contains("LOJAS ESPECIALIZADAS")')

fc.testes_iterativos(temp_df)

         grupo converteu    spending             transacoes         
         count      mean        mean         sum       mean      sum
grupo                                                               
alvo      5816  0.647008  379.611836  1428479.34   4.654797  17516.0
controle   656  0.599085  404.861883   159110.72   4.554707   1790.0
_____________________
Lift da conversão: 8.0%
Estatística t do teste de diferença de proporções: 2.43
P-valor do teste de diferença de proporções: 0.015209
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: -6.24%
Estatística t do teste de spending: 0.15
P-valor do teste de spending: 0.881714
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 2.2%
Estatística t do teste de frequencia: 1.71
P-valor do teste de frequencia: 0.087681
A diferença de frequência não é sig

In [44]:

temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("Mulher, 41 anos ou mais, Centro-Oeste e Norte")') \
        .query('segmento_mcc_agg.str.contains("SAUDE")') \
        .query('fx_ie.str.contains("alto")')

fc.testes_iterativos(temp_df)


         grupo converteu    spending            transacoes        
         count      mean        mean        sum       mean     sum
grupo                                                             
alvo      1496  0.690508  430.594966  444804.60   5.002904  5168.0
controle   157  0.592357  334.083011   31069.72   4.537634   422.0
_____________________
Lift da conversão: 16.57%
Estatística t do teste de diferença de proporções: 2.51
P-valor do teste de diferença de proporções: 0.012036
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 28.89%
Estatística t do teste de spending: 2.09
P-valor do teste de spending: 0.036362
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 10.25%
Estatística t do teste de frequencia: 2.17
P-valor do teste de frequencia: 0.030358
A diferença de frequência é 

In [31]:
temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("26 - Mulher, 25 a 40 anos")') \
        .query('segmento_mcc_agg.str.contains("SAUDE")') \
        .query('fx_ie.str.contains("alto")')

fc.testes_iterativos(temp_df)

         grupo converteu    spending            transacoes         
         count      mean        mean        sum       mean      sum
grupo                                                              
alvo      3881  0.686679  367.044221  978172.85   4.719700  12578.0
controle   449  0.657016  356.156373  105066.13   4.294915   1267.0
_____________________
Lift da conversão: 4.51%
Estatística t do teste de diferença de proporções: 1.28
P-valor do teste de diferença de proporções: 0.200799
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 3.06%
Estatística t do teste de spending: 0.79
P-valor do teste de spending: 0.428094
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 9.89%
Estatística t do teste de frequencia: 2.06
P-valor do teste de frequencia: 0.039781
A diferença de frequência é estatisticamente signifi

In [32]:
temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("Mulher")') \
        .query('segmento_mcc_agg.str.contains("LOJAS ESPECIALIZADAS")') \
        .query('fx_ie.str.contains("alto")')

fc.testes_iterativos(temp_df)

          grupo converteu    spending             transacoes          
          count      mean        mean         sum       mean       sum
grupo                                                                 
alvo      33630  0.685846  330.243574  7617068.03   4.440277  102415.0
controle   3732  0.677117  332.277305   839664.75   4.345865   10982.0
_____________________
Lift da conversão: 1.29%
Estatística t do teste de diferença de proporções: 1.09
P-valor do teste de diferença de proporções: 0.276108
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: -0.61%
Estatística t do teste de spending: 0.21
P-valor do teste de spending: 0.835875
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 2.17%
Estatística t do teste de frequencia: 1.57
P-valor do teste de frequencia: 0.117171
A diferença de frequência não é sign

In [33]:
temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("Mulher")') \
        .query('segmento_mcc_agg.str.contains("LOJAS DE DEPARTAMENTO")')
        #.query('fx_ie.str.contains("alto")')

fc.testes_iterativos(temp_df)

           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      107363  0.504569  361.376869  19576507.74   2.880713  156054.0
controle   11958  0.507025  359.175189   2177679.17   2.812469   17052.0
_____________________
Lift da conversão: -0.48%
Estatística t do teste de diferença de proporções: -0.51
P-valor do teste de diferença de proporções: 0.610383
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: 0.61%
Estatística t do teste de spending: 0.05
P-valor do teste de spending: 0.958667
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 2.43%
Estatística t do teste de frequencia: 1.15
P-valor do teste de frequencia: 0.248146
A diferença de frequência

In [34]:
temp_df = ativos_v1_p[ativos_v1_p['ds_cluster_sociodemographic'].isna() == False] \
        .query('ds_cluster_sociodemographic.str.contains("Mulher, 25 a 40 anos")') \
        .query('fx_ie.str.contains("alto")') \
        .query('segmento_mcc_agg.str.contains("SAUDE")')
        
fc.testes_iterativos(temp_df)

          grupo converteu    spending             transacoes         
          count      mean        mean         sum       mean      sum
grupo                                                                
alvo      26318  0.699293  383.475675  7057486.33   4.476255  82381.0
controle   2973  0.689876  384.323281   788247.05   4.472452   9173.0
_____________________
Lift da conversão: 1.37%
Estatística t do teste de diferença de proporções: 1.06
P-valor do teste de diferença de proporções: 0.288935
Conclusão:
A diferença de conversão não é significativa
______________________________________________________________
Lift do spending: -0.22%
Estatística t do teste de spending: 0.32
P-valor do teste de spending: 0.751394
Conclusão:
A diferença de spending não é significativa
______________________________________________________________
Lift da frequência: 0.09%
Estatística t do teste de frequencia: 0.6
P-valor do teste de frequencia: 0.546714
A diferença de frequência não é significat

Função de lift da campanha 
(leva 20 min)

In [36]:
#fc.analise_campanhas('flavia-costa', '2023-03-17', '2023-03-27', 'cpf', ativos_v1)

## ATIVOS - Sem comunicação x MCC

In [38]:
ativos_v2 = ativos.query('grupo_comunicacao != "Comunicação Genérica"')
ativos_v2['grupo'] = ['alvo' if x == 'Comunicação MCC' else 'controle' for x in ativos_v2['grupo_comunicacao']]

ativos_v2_c = fc.traz_conversao('flavia-costa', '2023-03-17', '2023-03-24', 'cpf', ativos_v2)

In [39]:
fc.testes_iterativos(ativos_v2_c)

            grupo converteu    spending               transacoes           
            count      mean        mean           sum       mean        sum
grupo                                                                      
alvo      1404826  0.488066  337.802139  2.316134e+08   3.467613  2377562.0
controle   173079  0.482080  330.341981  2.756307e+07   3.436036   286696.0
_____________________
Lift da conversão: 1.24%
Estatística t do teste de diferença de proporções: 4.7
P-valor do teste de diferença de proporções: 3e-06
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 2.26%
Estatística t do teste de spending: 4.92
P-valor do teste de spending: 1e-06
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 0.92%
Estatística t do teste de frequencia: 4.76
P-valor do teste de frequencia: 2e

In [40]:
ativos_v2_c['fx_ie'] = pd.qcut(ativos_v2_c['nr_score_ie'].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
ativos_v2_c['fx_ie'].value_counts()

baixo    522365
alto     522365
medio    522364
Name: fx_ie, dtype: int64

In [41]:
for x in ['baixo', 'medio', 'alto']:
    temp_df = pd.DataFrame()
    print(f"=============Faixa de IE: {x}============")
    temp_df = ativos_v2_c[ativos_v2_c['fx_ie'] == x]
    fc.testes_iterativos(temp_df)

=============Faixa de IE: baixo============
           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      464841  0.332109  330.369479  51001779.41   2.732747  421876.0
controle   57524  0.323169  327.454598   6087380.97   2.669231   49621.0
_____________________
Lift da conversão: 2.77%
Estatística t do teste de diferença de proporções: 4.3
P-valor do teste de diferença de proporções: 1.7e-05
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 0.89%
Estatística t do teste de spending: 2.02
P-valor do teste de spending: 0.043838
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 2.38%
Estatística t do teste de frequencia: 4.57


In [42]:
#por segmento MCC
for x in ['0 - OUTROS', '4 - SAUDE', '3 - SERVICOS DE ALIMENTACAO','2 - LOJAS DE DEPARTAMENTO','1 - SUPERMERCADO', '5 - LOJAS ESPECIALIZADAS']:
    temp_df = pd.DataFrame()
    print(f"=============Segmento MCC: {x}============")
    temp_df = ativos_v2_c[ativos_v2_c['segmento_mcc_agg'] == x]
    fc.testes_iterativos(temp_df)

=============Segmento MCC: 0 - OUTROS============
           grupo converteu    spending              transacoes          
           count      mean        mean          sum       mean       sum
grupo                                                                   
alvo      632263  0.392156  339.958517  84291354.42   2.994704  742525.0
controle   77896  0.384641  326.264469   9775536.01   2.953975   88507.0
_____________________
Lift da conversão: 1.95%
Estatística t do teste de diferença de proporções: 4.06
P-valor do teste de diferença de proporções: 5e-05
Conclusão:
A diferença de conversão é estatisticamente significativa
______________________________________________________________
Lift do spending: 4.2%
Estatística t do teste de spending: 4.69
P-valor do teste de spending: 3e-06
Conclusão:
A diferença de spending é estatisticamente significativa
______________________________________________________________
Lift da frequência: 1.38%
Estatística t do teste de frequencia: 4.14